In [158]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import warnings
import numpy as np
import scipy.stats as ss

from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler
from fancyimpute import KNN
from sklearn.impute import KNNImputer
from fancyimpute import IterativeImputer
from scipy.stats import chi2_contingency


In [120]:
data = pd.read_csv('../raw_data/dataset.csv',sep=';')
data = data.copy()
data.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,1,1,1,1,0,0,0,178839,9.653333,1.0
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,NaN
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1.0
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1.0
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,NaN


In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99976 entries, 0 to 99975
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   uuid                                 99976 non-null  object 
 1   default                              89976 non-null  float64
 2   account_amount_added_12_24m          99976 non-null  int64  
 3   account_days_in_dc_12_24m            88140 non-null  float64
 4   account_days_in_rem_12_24m           88140 non-null  float64
 5   account_days_in_term_12_24m          88140 non-null  float64
 6   account_incoming_debt_vs_paid_0_24m  40661 non-null  float64
 7   account_status                       45603 non-null  float64
 8   account_worst_status_0_3m            45603 non-null  float64
 9   account_worst_status_12_24m          33215 non-null  float64
 10  account_worst_status_3_6m            42274 non-null  float64
 11  account_worst_status_6_12m  

## 1. Preprocessing workflow 

In [122]:
# dropping the target variable
target = data['default']
data.drop(columns = ['default'], inplace=True)

### 1.1. Duplicates


In [123]:
# Duplicates
duplicate_count = data.duplicated().sum()
duplicate_count

0

 ### 1.2. Missing values 

In [124]:
# Missing data percentage
round((data.isnull().sum()/len(data)).sort_values(ascending=False),2)

worst_status_active_inv                0.70
account_worst_status_12_24m            0.67
account_worst_status_6_12m             0.60
account_incoming_debt_vs_paid_0_24m    0.59
account_worst_status_3_6m              0.58
account_worst_status_0_3m              0.54
account_status                         0.54
avg_payment_span_0_3m                  0.49
avg_payment_span_0_12m                 0.24
num_active_div_by_paid_inv_0_12m       0.23
num_arch_written_off_12_24m            0.18
num_arch_written_off_0_12m             0.18
account_days_in_rem_12_24m             0.12
account_days_in_term_12_24m            0.12
account_days_in_dc_12_24m              0.12
sum_paid_inv_0_12m                     0.00
sum_capital_paid_account_12_24m        0.00
sum_capital_paid_account_0_12m         0.00
time_hours                             0.00
status_max_archived_0_24_months        0.00
status_max_archived_0_12_months        0.00
status_max_archived_0_6_months         0.00
status_3rd_last_archived_0_24m  

In [125]:
_ = round((data.isnull().sum()/len(data)).sort_values(ascending=False),2)
miss_val_features_df = pd.DataFrame(_,columns=['percentage']).reset_index(names=['features'])
miss_val_features_df[miss_val_features_df['percentage'] >= 0.01]

,features,percentage
0,worst_status_active_inv,0.70
1,account_worst_status_12_24m,0.67
2,account_worst_status_6_12m,0.60
3,account_incoming_debt_vs_paid_0_24m,0.59
4,account_worst_status_3_6m,0.58
5,account_worst_status_0_3m,0.54
6,account_status,0.54
7,avg_payment_span_0_3m,0.49
8,avg_payment_span_0_12m,0.24
9,num_active_div_by_paid_inv_0_12m,0.23


### 1.3. Detect the MCAR pattern

> #### Detecting a MCAR pattern we can use statistical tests.
> #### We can use the qui-square test to test the goodness of fit, homogenity and independence 
> #### let's the first feature "worst_status_active_inv"
> #### alpha = 0.05 - the critical value 
> #### H_0: there is no association the chosen feature and the other ones


In [126]:
# The column is false
# data["account_incoming_debt_vs_paid_0_24m"] = False
# Replace where Height_missing with True where Height is missing
# data.loc[data[data["account_incoming_debt_vs_paid_0_24m"].isnull()].index, "account_incoming_debt_vs_paid_0_24m"] == True
data.loc[data["account_incoming_debt_vs_paid_0_24m"].isnull(), "account_incoming_debt_vs_paid_0_24m"] = True
data.loc[data["account_incoming_debt_vs_paid_0_24m"] != True, "account_incoming_debt_vs_paid_0_24m"] = False

> #### Let's see if the missing values of "account_worst_status_12_24m" are correlated with "sum_paid_inv_0_12m", which does not have missing values 

In [127]:
_ = data[data["account_incoming_debt_vs_paid_0_24m"]==False].groupby("merchant_group")["account_incoming_debt_vs_paid_0_24m"].count().value_counts().keys()
false_list = np.array(_)
false_list

array([  305,  2456,  8717,  1274, 17304,   308,   165,  3194,  1601,
         432,   536,  4365])

In [128]:
true_list = data[data["account_incoming_debt_vs_paid_0_24m"]==True].groupby("merchant_group")["account_incoming_debt_vs_paid_0_24m"].count().value_counts().keys()
true_list
# if len(true_list) == 0:
#    true_list = np.array(len(true_list))
#    true_list = np.zeros(false_list.shape)
# else:
#    true_list = np.array(true_list)

Int64Index([632, 2652, 8011, 1760, 31475, 439, 199, 4162, 2117, 690, 522,
            6660],
           dtype='int64')

In [129]:
# primeiro passo
# tenho de fazer o seguinte para as listas falsas e verdadeiras
# mesmo que seja zero a count é necessário que apareça 
# true_list e false_list têm de ter a mesma shape, sempre. 
# segundo passo 
# obter o p-value para todas as categorias

In [130]:
table = []
table.append(true_list)
table.append(false_list)


In [131]:
from scipy.stats import chi2_contingency
chi2, p, dof, ex = chi2_contingency(table)

In [132]:
print("The p-value is esqual to {}".format(p))

The p-value is esqual to 0.0


> #### Conclusion: We reject the null hypothesys. So, we **CANNOT** say there is **no association** they're independent

> #### Lets do the same for all features 

In [133]:
# selecting only the missing values features 
missing_features_only = miss_val_features_df[miss_val_features_df['percentage'] >= 0.1]['features']
missing_features_only


0                 worst_status_active_inv
1             account_worst_status_12_24m
2              account_worst_status_6_12m
3     account_incoming_debt_vs_paid_0_24m
4               account_worst_status_3_6m
5               account_worst_status_0_3m
6                          account_status
7                   avg_payment_span_0_3m
8                  avg_payment_span_0_12m
9        num_active_div_by_paid_inv_0_12m
10            num_arch_written_off_12_24m
11             num_arch_written_off_0_12m
12             account_days_in_rem_12_24m
13            account_days_in_term_12_24m
14              account_days_in_dc_12_24m
Name: features, dtype: object

In [134]:
# analysing these missing values features
# below one can see the "account...status" are discrete
for i in missing_features_only[0:6+1]:
    print(data[missing_features_only][i].unique())

[ 1. nan  2.  3.]
[nan  1.  2.  3.  4.]
[nan  1.  3.  2.  4.]
[False True]
[ 1. nan  2.  3.  4.]
[ 1. nan  2.  3.  4.]
[ 1. nan  2.  3.  4.]


In [135]:
# as we can see the next some of the other features are also discrete
for i in missing_features_only[7:]:
    print(len(data[missing_features_only][i].unique()))
    

2257
7940
862
4
3
283
65
128


In [136]:
# according to what we said in "Dataset_explanation.ipynb" 
# these features means "number of archieved invoices that were unpaid between M and N months ago"
# I am attempted to consider them as discrete feature given the values we have
# probably, we have an upper limit for this situation. Meaning, customers are not allowed to have more than 2 unpaid invoices for a period of 24 months
# or 1 unpaid invoice for a period of 12 months
for i in missing_features_only[10:11+1]:
    print(data[missing_features_only][i].unique())

[ 0. nan  2.  1.]
[ 0. nan  1.]


In [137]:
# lets go further on this 
data[['uuid','num_arch_written_off_0_12m', 'num_arch_written_off_12_24m']][(data['num_arch_written_off_12_24m']  > 0) | (data['num_arch_written_off_0_12m'] > 0)]

,uuid,num_arch_written_off_0_12m,num_arch_written_off_12_24m
4736,fe583a43-ee49-4268-8ce5-1b1c53c38a0a,1.0,0.0
11939,0acd2796-3f77-4ce7-97ff-e3443d528fc7,1.0,0.0
19795,9b254660-b084-40b3-8e53-a2d11081f945,0.0,2.0
25071,08c0505e-a202-4627-970c-0e400af3807a,1.0,0.0
30081,cfb183fc-a502-408d-815d-cb9a4da3e110,0.0,1.0
44171,911b27da-977a-48bf-9354-548d3a52789e,0.0,1.0
50262,531f5e8e-b59b-4944-8215-4ba474a51bd2,1.0,0.0
54864,15d51024-2f68-4394-aa29-a909ed11609b,1.0,0.0
59024,9176cb03-325b-483e-9ffa-927358b782fe,0.0,1.0
59630,261e4f5e-724e-4540-b24b-73dfe1616578,1.0,0.0


In [138]:
# our discrete variables are 
discrete_missing_values = ['account_worst_status_12_24m','account_worst_status_6_12m',
                           'account_worst_status_3_6m','account_worst_status_0_3m',
                           'account_status','num_arch_written_off_12_24m',
                           'num_arch_written_off_0_12m','default']
discrete_missing_values

['account_worst_status_12_24m',
 'account_worst_status_6_12m',
 'account_worst_status_3_6m',
 'account_worst_status_0_3m',
 'account_status',
 'num_arch_written_off_12_24m',
 'num_arch_written_off_0_12m',
 'default']

In [139]:
continuos_missing_values = list(set(missing_features_only).difference(discrete_missing_values))
continuos_missing_values

['worst_status_active_inv',
 'account_days_in_term_12_24m',
 'num_active_div_by_paid_inv_0_12m',
 'account_days_in_rem_12_24m',
 'avg_payment_span_0_3m',
 'avg_payment_span_0_12m',
 'account_days_in_dc_12_24m',
 'account_incoming_debt_vs_paid_0_24m']

### 1.3.1. Feature Binning

In [140]:
def quantile(numeric_feature):
    
    numeric_feature = [float(x) for x in numeric_feature] # convert all values to float
    
    q1 = pd.Series(numeric_feature).quantile(0.25)
    q2 = pd.Series(numeric_feature).quantile(0.50)
    q3 = pd.Series(numeric_feature).quantile(0.75)
    
    for i in range(len(numeric_feature)):
        if numeric_feature[i] <= q1:
            numeric_feature[i] = '1st_quant'
        elif (numeric_feature[i] > q1) and (numeric_feature[i] <= q2):
            numeric_feature[i] = '2nd_quant'
        elif (numeric_feature[i] > q2) and (numeric_feature[i] <= q3):
            numeric_feature[i] = '3rd_quant'  
        else: 
            numeric_feature[i] = '4th_quant'
    return np.array(numeric_feature)

In [142]:
data = { i: quantile(data[i]) if i in continuos_missing_values else data[i] for i in data }


In [143]:
# Creates DataFrame.
data = pd.DataFrame(data)
data.head()

,uuid,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,account_worst_status_3_6m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0,1st_quant,1st_quant,1st_quant,1st_quant,1.0,1.0,NaN,1.0,...,1,1,1,1,0,0,0,178839,9.653333,1st_quant
1,0e961183-8c15-4470-9a5e-07a1bd207661,0,1st_quant,1st_quant,1st_quant,2nd_quant,1.0,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,4th_quant
2,d8edaae6-4368-44e0-941e-8328f203e64e,0,1st_quant,1st_quant,1st_quant,2nd_quant,NaN,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1st_quant
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0,4th_quant,4th_quant,4th_quant,2nd_quant,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1st_quant
4,c8f8b835-5647-4506-bf15-49105d8af30b,0,1st_quant,1st_quant,1st_quant,2nd_quant,NaN,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,4th_quant


In [144]:
data_missing_only = {feature: data[feature] for feature in missing_features_only}
data_missing_only = pd.DataFrame(data_missing_only)
data_missing_only

,worst_status_active_inv,account_worst_status_12_24m,account_worst_status_6_12m,account_incoming_debt_vs_paid_0_24m,account_worst_status_3_6m,account_worst_status_0_3m,account_status,avg_payment_span_0_3m,avg_payment_span_0_12m,num_active_div_by_paid_inv_0_12m,num_arch_written_off_12_24m,num_arch_written_off_0_12m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_days_in_dc_12_24m
0,1st_quant,NaN,NaN,1st_quant,1.0,1.0,1.0,1st_quant,2nd_quant,4th_quant,0.0,0.0,1st_quant,1st_quant,1st_quant
1,4th_quant,1.0,1.0,2nd_quant,1.0,1.0,1.0,4th_quant,4th_quant,1st_quant,0.0,0.0,1st_quant,1st_quant,1st_quant
2,1st_quant,NaN,NaN,2nd_quant,NaN,NaN,NaN,3rd_quant,3rd_quant,3rd_quant,0.0,0.0,1st_quant,1st_quant,1st_quant
3,1st_quant,NaN,NaN,2nd_quant,NaN,NaN,NaN,1st_quant,1st_quant,3rd_quant,0.0,0.0,4th_quant,4th_quant,4th_quant
4,4th_quant,NaN,NaN,2nd_quant,NaN,NaN,NaN,2nd_quant,2nd_quant,1st_quant,0.0,0.0,1st_quant,1st_quant,1st_quant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99971,4th_quant,NaN,NaN,2nd_quant,NaN,1.0,1.0,4th_quant,1st_quant,1st_quant,0.0,0.0,1st_quant,1st_quant,1st_quant
99972,4th_quant,NaN,1.0,1st_quant,1.0,1.0,1.0,4th_quant,4th_quant,1st_quant,0.0,0.0,1st_quant,1st_quant,1st_quant
99973,4th_quant,2.0,1.0,1st_quant,1.0,2.0,2.0,4th_quant,4th_quant,4th_quant,NaN,NaN,4th_quant,1st_quant,1st_quant
99974,4th_quant,1.0,2.0,1st_quant,2.0,2.0,1.0,4th_quant,3rd_quant,1st_quant,0.0,0.0,1st_quant,1st_quant,1st_quant


In [145]:
# Transforming all missing values to "True"    
data_missing_only = data_missing_only.isna().applymap(lambda x: True if x else False)


In [146]:
data_missing_only

,worst_status_active_inv,account_worst_status_12_24m,account_worst_status_6_12m,account_incoming_debt_vs_paid_0_24m,account_worst_status_3_6m,account_worst_status_0_3m,account_status,avg_payment_span_0_3m,avg_payment_span_0_12m,num_active_div_by_paid_inv_0_12m,num_arch_written_off_12_24m,num_arch_written_off_0_12m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_days_in_dc_12_24m
0,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,True,True,False,True,True,True,False,False,False,False,False,False,False,False
3,False,True,True,False,True,True,True,False,False,False,False,False,False,False,False
4,False,True,True,False,True,True,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99971,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False
99972,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
99973,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False
99974,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [161]:
def getCombinations(seq):
    combinations = list()
    for i in range(0,len(seq)):
        for j in range(i+1,len(seq)):
            combinations.append([seq[i],seq[j]])
    return pd.DataFrame(combinations, columns = ['feature_1', 'feature_2'])

# source: https://stackoverflow.com/questions/20762574/combinations-with-two-elements


In [200]:
all_combinations = getCombinations(data_missing_only.columns)
all_combinations[10:15]

,feature_1,feature_2
10,worst_status_active_inv,num_arch_written_off_0_12m
11,worst_status_active_inv,account_days_in_rem_12_24m
12,worst_status_active_inv,account_days_in_term_12_24m
13,worst_status_active_inv,account_days_in_dc_12_24m
14,account_worst_status_12_24m,account_worst_status_6_12m


In [191]:
confusion_matrix = []
for index in range(0, len(all_combinations)):
    confusion_matrix.append(pd.crosstab(data_missing_only[all_combinations['feature_1'][index]], data_missing_only[all_combinations['feature_2'][index]]))

In [221]:
all_combinations['p_value'] = [ss.chi2_contingency(confusion_matrix[index])[1] for index in range(len(all_combinations))]
all_combinations


,feature_1,feature_2,chi_sq,p_value
0,worst_status_active_inv,account_worst_status_12_24m,0.0,1.0
1,worst_status_active_inv,account_worst_status_6_12m,0.0,1.0
2,worst_status_active_inv,account_incoming_debt_vs_paid_0_24m,0.0,1.0
3,worst_status_active_inv,account_worst_status_3_6m,0.0,1.0
4,worst_status_active_inv,account_worst_status_0_3m,0.0,1.0
...,...,...,...,...
100,num_arch_written_off_0_12m,account_days_in_term_12_24m,0.0,1.0
101,num_arch_written_off_0_12m,account_days_in_dc_12_24m,0.0,1.0
102,account_days_in_rem_12_24m,account_days_in_term_12_24m,0.0,1.0
103,account_days_in_rem_12_24m,account_days_in_dc_12_24m,0.0,1.0


In [237]:
all_combinations['p_value'][(all_combinations['p_value']>0) & (all_combinations['p_value']<1)]

Series([], Name: p_value, dtype: float64)

In [244]:
all_combinations['p_value'][all_combinations['p_value']>0]

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
100    1.0
101    1.0
102    1.0
103    1.0
104    1.0
Name: p_value, Length: 84, dtype: float64

In [214]:
#lets investigate these numbers 
matrix = pd.crosstab(data_missing_only['account_worst_status_12_24m'], data_missing_only['account_worst_status_6_12m'])
print(matrix)
ss.chi2_contingency(matrix)

account_worst_status_6_12m   False  True 
account_worst_status_12_24m              
False                        33215      0
True                          6411  60350


Chi2ContingencyResult(statistic=75749.98632703112, pvalue=0.0, dof=1, expected_freq=array([[13164.93548452, 20050.06451548],
       [26461.06451548, 40299.93548452]]))

In [239]:
#lets investigate these numbers 
matrix = pd.crosstab(data_missing_only['num_arch_written_off_0_12m'], data_missing_only['account_days_in_term_12_24m'])
print(matrix)
ss.chi2_contingency(matrix)

account_days_in_term_12_24m  False
num_arch_written_off_0_12m        
False                        81898
True                         18078


Chi2ContingencyResult(statistic=0.0, pvalue=1.0, dof=0, expected_freq=array([[81898.],
       [18078.]]))

In [250]:
data['account_days_in_term_12_24m'].isnull().sum()
# if does not have missing values I have to take this out from missing values list
# all features must have True and False values 

0

In [ ]:
# 1 read chatgpt -  code explanations 
# 2 read this article and apply the aboe to it 
# 3 please!!! I have to start the imputation



In [148]:
def getCombinations(seq):
    combinations = list()
    for i in range(0,len(seq)):
        for j in range(i+1,len(seq)):
            combinations.append([seq[i],seq[j]])
    return combinations

# source: https://stackoverflow.com/questions/20762574/combinations-with-two-elements


In [149]:
# Creates DataFrame
unique_combinations = pd.DataFrame(getCombinations(data.columns), columns = ['feature_1', 'feature_2'])
unique_combinations

,feature_1,feature_2
0,uuid,account_amount_added_12_24m
1,uuid,account_days_in_dc_12_24m
2,uuid,account_days_in_rem_12_24m
3,uuid,account_days_in_term_12_24m
4,uuid,account_incoming_debt_vs_paid_0_24m
...,...,...
856,sum_capital_paid_account_12_24m,time_hours
857,sum_capital_paid_account_12_24m,worst_status_active_inv
858,sum_paid_inv_0_12m,time_hours
859,sum_paid_inv_0_12m,worst_status_active_inv


In [150]:
data['account_amount_added_12_24m'].unique()

array([     0,  57229, 148922, ..., 112441,  88405,  45671])

In [151]:

confusion_matrix = pd.crosstab(data['sum_capital_paid_account_12_24m'], data['account_amount_added_12_24m'])

KeyboardInterrupt: 

In [192]:
## Function to find Cramers coeff for categorical variables
import scipy.stats as ss

def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

# source: https://towardsdatascience.com/statistical-test-for-mcar-in-python-9fb617a76eac
# source: https://stackoverflow.com/questions/20892799/using-pandas-calculate-cram%C3%A9rs-coefficient-matrix

In [ ]:
# 1 tenho de ler stackoverflow - 2nd link
# 2 saber como funciona "crosstab"
# 3 ler 1st Medium link
# 4 tentar obter as relaçoes entre features 


In [184]:
cramers_corrected_stat(confusion_matrix)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (105,) + inhomogeneous part.